# Demo SequenceActivity
The sequence activity calls all activities which are specified as sub processes.

In [6]:
import datetime, time
import simpy

import pandas as pd
import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot

# setup environment
simulation_start = 0
my_env = simpy.Environment(initial_time=simulation_start)
registry = {}

## Definition of the subprocesses used in the seqeunce activity
The sequence activity executes three basic activities in a sequence. Since the individual basic activities are executed one after the other, it is necessary to postpone the initialization of the initialization of these activities. This is done using the **postpone_start** parameter.

To ensure that all logging of the basic activities are available in a single log, additional logging is configured to an reporting activity.

In [2]:
reporting_activity_data = {
    "env": my_env,
    "name": "Reporting activity",
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff5k",  # For logging purposes
    "registry": registry,
    "duration": 0,
    "postpone_start": False,
}
reporting_activity = model.BasicActivity(**reporting_activity_data)

sub_processes = []
basic_activity_data1 = {
    "env": my_env,
    "name": "Basic activity1",
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff5b",  # For logging purposes
    "registry": registry,
    "duration": 14,
    "postpone_start": True,
    "additional_logs": [reporting_activity],
}
sub_processes.append(model.BasicActivity(**basic_activity_data1))
basic_activity_data2 = {
    "env": my_env,
    "name": "Basic activity2",
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff5c",  # For logging purposes
    "registry": registry,
    "duration": 5,
    "additional_logs": [reporting_activity],
    "postpone_start": True,
}
sub_processes.append(model.BasicActivity(**basic_activity_data2))
basic_activity_data3 = {
    "env": my_env,
    "name": "Basic activity3",
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff5d",  # For logging purposes
    "registry": registry,
    "duration": 220,
    "additional_logs": [reporting_activity],
    "postpone_start": True,
}
sub_processes.append(model.BasicActivity(**basic_activity_data3))



## Definition of seqeunce activity

In [3]:
sequential_activity_data = {
    "env": my_env,
    "name": "Sequential process",
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff60",  # For logging purposes
    "registry": registry,
    "sub_processes": sub_processes,
}
activity = model.SequentialActivity(**sequential_activity_data)

In [4]:
my_env.run()

In [11]:
display(plot.get_log_dataframe(reporting_activity, [*sub_processes, activity, reporting_activity]))
for act in [*sub_processes, activity]:
    display(plot.get_log_dataframe(act, [*sub_processes, activity, reporting_activity]))

,Timestamp,Activity,ActivityState
0,1970-01-01 00:00:00,Reporting activity,START
1,1970-01-01 00:00:00,Basic activity1,START
2,1970-01-01 00:00:00,Reporting activity,STOP
3,1970-01-01 00:00:14,Basic activity1,STOP
4,1970-01-01 00:00:14,Basic activity2,START
5,1970-01-01 00:00:19,Basic activity2,STOP
6,1970-01-01 00:00:19,Basic activity3,START
7,1970-01-01 00:03:59,Basic activity3,STOP


,Timestamp,Activity,ActivityState
0,1970-01-01 00:00:00,Basic activity1,START
1,1970-01-01 00:00:14,Basic activity1,STOP


,Timestamp,Activity,ActivityState
0,1970-01-01 00:00:14,Basic activity2,START
1,1970-01-01 00:00:19,Basic activity2,STOP


,Timestamp,Activity,ActivityState
0,1970-01-01 00:00:19,Basic activity3,START
1,1970-01-01 00:03:59,Basic activity3,STOP


,Timestamp,Activity,ActivityState
0,1970-01-01 00:00:00,Sequential process,START
1,1970-01-01 00:00:00,Sequential process,START
2,1970-01-01 00:00:14,Sequential process,STOP
3,1970-01-01 00:00:14,Sequential process,START
4,1970-01-01 00:00:19,Sequential process,STOP
5,1970-01-01 00:00:19,Sequential process,START
6,1970-01-01 00:03:59,Sequential process,STOP
7,1970-01-01 00:03:59,Sequential process,STOP
